In [1]:
%pylab inline
import numpy as np
import torch
import os

from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.autograd import Variable
from torch.utils.data import dataloader

from torchvision import transforms
from torchvision.datasets import MNIST, FashionMNIST

from sklearn.metrics import accuracy_score
from collections import deque

from graphviz import Digraph

Populating the interactive namespace from numpy and matplotlib


# Configuration

In [2]:
BATCH_SIZE = 768

# Data

In [3]:
train = FashionMNIST('./data', train=True, download=True, transform=transforms.Compose([
    transforms.ToTensor(), # ToTensor does min-max normalization. 
]), )

test = FashionMNIST('./data', train=False, download=True, transform=transforms.Compose([
    transforms.ToTensor(), # ToTensor does min-max normalization. 
]), )

# Create DataLoader
dataloader_args = dict(shuffle=True, batch_size=BATCH_SIZE ,num_workers=1, pin_memory=True)
train_loader = dataloader.DataLoader(train, **dataloader_args)
test_loader = dataloader.DataLoader(test, **dataloader_args)

In [4]:
def visualize_data(data):
    N = data.size()[0]
    
    fig, subplots = plt.subplots(5, 10)
    fig.set_figheight(6)
    fig.set_figwidth(10)
    subplots = subplots.reshape(-1)
    
    for subplot in subplots:
        idx = np.random.randint(0, N)
        img = train.train_data[idx].numpy()
        
        subplot.get_xaxis().set_visible(False)
        subplot.get_yaxis().set_visible(False)
        subplot.imshow(img, cmap='gray')
        
visualize_data(train.train_data)

# Model

## Discriminator

In [5]:
np.random.seed(0)
torch.manual_seed(0)

class Discriminator(nn.Module):
    def __init__(self, channel:int=256):
        super(Discriminator, self).__init__()
        np.random.seed(0)
        torch.manual_seed(0)
        
        self.channel:int = channel
        n_class:int = 1 # 1=진짜, 0=가짜 이기 때문에 1값으로 둔다.
        
        self.conv1 = nn.Conv2d(1, channel//4, kernel_size=6, stride=1, padding=2)
        self.conv2 = nn.Conv2d(channel//4, channel//3, kernel_size=5, stride=2, padding=4)
        self.bn2 = nn.BatchNorm2d(channel//3)
        self.conv3 = nn.Conv2d(channel//3, channel//2, kernel_size=4, stride=2, padding=4)
        self.bn3 = nn.BatchNorm2d(channel//2)
        self.conv4 = nn.Conv2d(channel//2, channel, kernel_size=4, stride=2, padding=4)
        self.bn4 = nn.BatchNorm2d(channel)
        
        # Global Average Pooling
        self.conv5 = nn.Conv2d(channel, n_class, kernel_size=4, stride=2, padding=4)

    def forward(self, x, _return_activations=False):
        h1: Variable = F.leaky_relu(self.conv1(x), negative_slope=0.2)
        h2: Variable = F.leaky_relu(self.bn2(self.conv2(h1)), negative_slope=0.2)
        h3: Variable = F.leaky_relu(self.bn3(self.conv3(h2)), negative_slope=0.2)
        h4: Variable = F.leaky_relu(self.bn4(self.conv4(h3)), negative_slope=0.2)
        
        # Global Average Pooling
        h5 = F.leaky_relu(self.conv5(h4), negative_slope=0.2)
        gap = F.avg_pool2d(h5, kernel_size=h5.size()[2:]).view(h5.size()[0], -1)
        output = F.sigmoid(gap)
        
        if _return_activations:
            return output, [h1, h2, h3, h4, h5, gap]
        
        return output
    
    def feature_mapping(self, real, fake):
        _, real_fs = self.forward(real, _return_activations=True)
        _, fake_fs = self.forward(fake, _return_activations=True)
        real = real_fs
        fake = fake_fs
        
        diffs = []
        for real, fake in zip(real_fs, fake_fs):
            diff = torch.sqrt(torch.mean(torch.pow(real - fake, 2)))
            diffs.append(diff)
        
        output = torch.mean(torch.stack(diffs))
        
#         print('x'*50)
#         output.backward()
#         print(fake.grad_fn)
#         print(dir(fake.grad_fn.next_functions[0][0]))
        return output

    def residual_loss(self, real_y, generated_y):
        return torch.sum(torch.abs(real_y - generated_y))
    
    def improved_loss(self, real_y, generated_y, l=0.5):
        residual = self.residual_loss(real_y, generated_y)
        d_loss = self.feature_mapping(real_y, generated_y)
        return (1-l) * residual + l * d_loss
        
D = Discriminator()
print(D.cuda()) # CUDA!
d_optimizer = optim.Adam(D.parameters(), lr=0.001)



# g_optimizer.zero_grad()
# real = real_images[0:2]
# noise = Variable(torch.randn((2, 100, 1, 1)).cuda())
# fake = G(noise)
# D.improved_loss(real, fake)

# D.feature_mapping(real, fake)
# print()

Discriminator (
  (conv1): Conv2d(1, 64, kernel_size=(6, 6), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(64, 85, kernel_size=(5, 5), stride=(2, 2), padding=(4, 4))
  (bn2): BatchNorm2d(85, eps=1e-05, momentum=0.1, affine=True)
  (conv3): Conv2d(85, 128, kernel_size=(4, 4), stride=(2, 2), padding=(4, 4))
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
  (conv4): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(4, 4))
  (bn4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
  (conv5): Conv2d(256, 1, kernel_size=(4, 4), stride=(2, 2), padding=(4, 4))
)


## Generator

In [6]:
np.random.seed(0)
torch.manual_seed(0)

class Generator(nn.Module):
    def __init__(self, noise:int=100, channel:int=64):
        super(Generator, self).__init__()
        np.random.seed(0)
        torch.manual_seed(0)
        _c = channel
        
        self.noise = noise
        self.tp_conv1 = nn.ConvTranspose2d(noise, _c*16, kernel_size=4, stride=1, padding=0, bias=False)
        self.bn1 = nn.BatchNorm2d(_c*16)
        
        self.tp_conv2 = nn.ConvTranspose2d(_c*16, _c*8, kernel_size=4, stride=2, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(_c*8)
        
        self.tp_conv3 = nn.ConvTranspose2d(_c*8, _c*4, kernel_size=4, stride=2, padding=1, bias=False)
        self.bn3 = nn.BatchNorm2d(_c*4)
        
        self.tp_conv4 = nn.ConvTranspose2d(_c*4, _c*2, kernel_size=4, stride=1, padding=2, bias=False)
        self.bn4 = nn.BatchNorm2d(_c*2)
        
        self.tp_conv5 = nn.ConvTranspose2d(_c*2, _c, kernel_size=4, stride=2, padding=2, bias=False)
        self.bn5 = nn.BatchNorm2d(_c)
        
        self.tp_conv6 = nn.ConvTranspose2d(_c, 1, kernel_size=5, stride=1, padding=2, bias=False)
    
    def forward(self, noise):
        h = self.tp_conv1(noise)
        h = F.relu(self.bn1(h))
#         print('h1', h.size())
        
        h = self.tp_conv2(h)
        h = F.relu(self.bn2(h))
#         print('h2', h.size())
        
        h = self.tp_conv3(h)
        h = F.relu(self.bn3(h))
#         print('h3', h.size())
        
        h = self.tp_conv4(h)
        h = F.relu(self.bn4(h))
#         print('h4', h.size())
        
        h = self.tp_conv5(h)
        h = F.relu(self.bn5(h))
#         print('h5', h.size())
        
        h = self.tp_conv6(h)
        h = F.sigmoid(h)
#         print('h6', h.size())
        return h
    

G = Generator()
print(G.cuda())
g_optimizer = optim.Adam(G.parameters(), lr=0.001)

Generator (
  (tp_conv1): ConvTranspose2d(100, 1024, kernel_size=(4, 4), stride=(1, 1), bias=False)
  (bn1): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True)
  (tp_conv2): ConvTranspose2d(1024, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
  (tp_conv3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
  (tp_conv4): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(1, 1), padding=(2, 2), bias=False)
  (bn4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
  (tp_conv5): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(2, 2), bias=False)
  (bn5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (tp_conv6): ConvTranspose2d(64, 1, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), bias=False)
)


## AnoGAN Functions

In [7]:
def residual_loss(real_y, generated_y):
    return torch.sum(torch.abs(real_y - generated_y))

def anomaly_loss(G, D, real_y, generated_y, l=0.5):
    loss_r = residual_loss(real_y, generated_y)
    loss_d = D.feature_mapping(real_y, generated_y)
    return (1-l) * loss_r + l * loss_d
    

# real = Variable(real_images[0:2].cuda())
# noise = Variable(torch.randn((2, 100, 1, 1)).cuda())
# fake = G(noise)

# anomaly_loss(G, D, real, fake)

## Save Model

In [8]:
def save_moodel(filename='anogan', epoch:int=None, divergence:float=None):
    if not os.path.exists('./checkpoints'):
        os.mkdir('./checkpoints')
        
    check_point = {'G': G.state_dict(), 
                   'D': D.state_dict(), 
                   'g_optimizer': g_optimizer.state_dict(),
                   'd_optimizer': d_optimizer.state_dict(),
                   'divergence': divergences, 
                   'd_losses': d_losses, 
                   'g_losses': g_losses, 
                   'd_fake_losses': d_fake_losses, 
                   'd_real_losses': d_real_losses}
    f_name = f'checkpoints/{filename}' 
    if epoch is not None:
        f_name += f'_{epoch}'
    if divergence is not None:
        f_name += f'_{divergence:<.4}'
    f_name += '.pth.tar'
    torch.save(check_point, f_name)
    print(f'checkpoint >> {f_name}')
# save_moodel()

# Train

In [ ]:
N_EPOCH = 300

real_y = Variable(torch.ones((BATCH_SIZE, 1)).cuda())
fake_y = Variable(torch.zeros((BATCH_SIZE, 1)).cuda())
loss_f = nn.BCELoss()

d_real_losses = list()
d_fake_losses = list()
d_losses = list()
g_losses = list()
divergences = list()

In [ ]:
# Train Discriminator with Generator not being trained
# 먼저 Discriminator를 학습시킵니다.
# 이때 real image와 fake이미지 두개의 데이터를 사용하여 학습합니다. 
# Discriminator를 학습시킬때는 Generator는 학습시키면 안됩니다.
for epoch in range(N_EPOCH):
    for step, (real_images, _) in enumerate(train_loader):
        # Samples
        _batch_size = real_images.size(0)
        real_images = Variable(real_images.cuda())
        z = Variable(torch.randn((_batch_size, 100)).cuda())
                
        ###############################################
        # Train D (But do not train G)
        ###############################################
        # Init D
        d_optimizer.zero_grad()
        
        # Calculate the loss with real images
        y_real_pred = D(real_images)
        d_real_loss = loss_f(y_real_pred, real_y[:_batch_size])
        
        # Calculate the loss with fake images
        noise = Variable(torch.randn((_batch_size, 100, 1, 1)).cuda())
        y_fake_pred = D(G(noise).detach())
        d_fake_loss = loss_f(y_fake_pred, fake_y[:_batch_size])
        
        # Update D with G not being updated        
        d_loss = d_real_loss + d_fake_loss
        d_loss.backward()
        d_optimizer.step()
        
        ###############################################
        # Train G with fake images but do not train G
        ###############################################
        g_optimizer.zero_grad()
        
        noise = Variable(torch.randn((_batch_size, 100, 1, 1)).cuda())
        
        g_loss = D.feature_mapping(real_images, G(noise))
        g_loss.backward()
        g_optimizer.step()
        
        ###############################################
        # Visualization
        ###############################################
        if step%5 == 0:
            d_real_losses.append(d_real_loss.data[0])
            d_fake_losses.append(d_fake_loss.data[0])
            d_losses.append(d_loss.data[0])
            g_losses.append(g_loss.data.cpu().numpy()[0])

            divergences.append(torch.mean(y_real_pred/(y_real_pred+y_fake_pred)).data[0])
        
        if step % 50 == 0:
            print(f'\r[{epoch+1}/{N_EPOCH}]',
                  # '{:.3}'.format(torch.mean(params[0]).data[0]),
                  'divergence: {:<8.3}'.format(np.mean(divergences[-100:])),
                  'D: {:<8.3}'.format(np.mean(d_fake_losses[-100:])), 
                  'D_real: {:<8.3}'.format(np.mean(d_real_losses[-100:])),
                  'D_fake: {:<8.3}'.format(np.mean(d_fake_losses[-100:])), 
                  'G:{:<8.3}'.format(np.mean(g_losses[-100:])), end='')
    print()
    save_moodel(epoch=epoch, divergence=np.mean(divergences[-100:]))
    print()

[1/300] divergence: 0.532    D: 0.697    D_real: 0.585    D_fake: 0.697    G:0.622   
checkpoint >> checkpoints/anogan_0_0.5278.pth.tar

[2/300] divergence: 0.521    D: 0.662    D_real: 0.662    D_fake: 0.662    G:0.6     
checkpoint >> checkpoints/anogan_1_0.5246.pth.tar

[3/300] divergence: 0.536    D: 0.652    D_real: 0.628    D_fake: 0.652    G:0.752   
checkpoint >> checkpoints/anogan_2_0.5391.pth.tar

[4/300] divergence: 0.54     D: 0.645    D_real: 0.623    D_fake: 0.645    G:0.826   
checkpoint >> checkpoints/anogan_3_0.5397.pth.tar

[5/300] divergence: 0.558    D: 0.613    D_real: 0.59     D_fake: 0.613    G:0.99    
checkpoint >> checkpoints/anogan_4_0.575.pth.tar

[6/300] divergence: 0.595    D: 0.558    D_real: 0.541    D_fake: 0.558    G:1.32    
checkpoint >> checkpoints/anogan_5_0.6093.pth.tar

[7/300] divergence: 0.656    D: 0.464    D_real: 0.466    D_fake: 0.464    G:2.14    
checkpoint >> checkpoints/anogan_6_0.6798.pth.tar

[8/300] divergence: 0.728    D: 0.37     D

[60/300] divergence: 0.901    D: 0.127    D_real: 0.126    D_fake: 0.127    G:4.43    
checkpoint >> checkpoints/anogan_59_0.8973.pth.tar

[61/300] divergence: 0.904    D: 0.124    D_real: 0.133    D_fake: 0.124    G:4.52    
checkpoint >> checkpoints/anogan_60_0.9029.pth.tar

[62/300] divergence: 0.894    D: 0.139    D_real: 0.149    D_fake: 0.139    G:4.44    
checkpoint >> checkpoints/anogan_61_0.8951.pth.tar

[63/300] divergence: 0.894    D: 0.138    D_real: 0.152    D_fake: 0.138    G:4.51    
checkpoint >> checkpoints/anogan_62_0.8946.pth.tar

[64/300] divergence: 0.899    D: 0.131    D_real: 0.168    D_fake: 0.131    G:4.39    
checkpoint >> checkpoints/anogan_63_0.9007.pth.tar

[65/300] divergence: 0.904    D: 0.121    D_real: 0.161    D_fake: 0.121    G:4.41    
checkpoint >> checkpoints/anogan_64_0.904.pth.tar

[66/300] divergence: 0.901    D: 0.124    D_real: 0.162    D_fake: 0.124    G:4.28    
checkpoint >> checkpoints/anogan_65_0.8999.pth.tar

[67/300] divergence: 0.897  

[119/300] divergence: 0.945    D: 0.0693   D_real: 0.0722   D_fake: 0.0693   G:5.3     
checkpoint >> checkpoints/anogan_118_0.9454.pth.tar

[120/300] divergence: 0.943    D: 0.0724   D_real: 0.0808   D_fake: 0.0724   G:5.36    
checkpoint >> checkpoints/anogan_119_0.9406.pth.tar

[121/300] divergence: 0.939    D: 0.0767   D_real: 0.0996   D_fake: 0.0767   G:5.16    
checkpoint >> checkpoints/anogan_120_0.9397.pth.tar

[122/300] divergence: 0.938    D: 0.0764   D_real: 0.0994   D_fake: 0.0764   G:5.06    
checkpoint >> checkpoints/anogan_121_0.9368.pth.tar

[123/300] divergence: 0.935    D: 0.0798   D_real: 0.104    D_fake: 0.0798   G:4.84    
checkpoint >> checkpoints/anogan_122_0.935.pth.tar

[124/300] divergence: 0.933    D: 0.0802   D_real: 0.118    D_fake: 0.0802   G:4.78    
checkpoint >> checkpoints/anogan_123_0.9315.pth.tar

[125/300] divergence: 0.936    D: 0.0742   D_real: 0.116    D_fake: 0.0742   G:4.77    
checkpoint >> checkpoints/anogan_124_0.9355.pth.tar

[126/300] dive

[177/300] divergence: 0.949    D: 0.0863   D_real: 0.0679   D_fake: 0.0863   G:5.42    
checkpoint >> checkpoints/anogan_176_0.9469.pth.tar

[178/300] divergence: 0.947    D: 0.0895   D_real: 0.0629   D_fake: 0.0895   G:5.42    
checkpoint >> checkpoints/anogan_177_0.9484.pth.tar

[179/300] divergence: 0.947    D: 0.0881   D_real: 0.0637   D_fake: 0.0881   G:5.38    
checkpoint >> checkpoints/anogan_178_0.9436.pth.tar

[180/300] divergence: 0.935    D: 0.119    D_real: 0.0602   D_fake: 0.119    G:5.38    
checkpoint >> checkpoints/anogan_179_0.9322.pth.tar

[181/300] divergence: 0.934    D: 0.12     D_real: 0.0655   D_fake: 0.12     G:5.42    
checkpoint >> checkpoints/anogan_180_0.9363.pth.tar

[182/300] divergence: 0.94     D: 0.0965   D_real: 0.0616   D_fake: 0.0965   G:5.38    
checkpoint >> checkpoints/anogan_181_0.9407.pth.tar

[183/300] divergence: 0.945    D: 0.0809   D_real: 0.052    D_fake: 0.0809   G:5.29    
checkpoint >> checkpoints/anogan_182_0.9432.pth.tar

[184/300] div

[235/300] divergence: 0.924    D: 0.104    D_real: 0.11     D_fake: 0.104    G:4.92    
checkpoint >> checkpoints/anogan_234_0.9267.pth.tar

[236/300] divergence: 0.927    D: 0.099    D_real: 0.107    D_fake: 0.099    G:4.91    
checkpoint >> checkpoints/anogan_235_0.9251.pth.tar

[237/300] divergence: 0.927    D: 0.0991   D_real: 0.109    D_fake: 0.0991   G:4.93    
checkpoint >> checkpoints/anogan_236_0.9266.pth.tar

[238/300] divergence: 0.935    D: 0.082    D_real: 0.119    D_fake: 0.082    G:4.94    
checkpoint >> checkpoints/anogan_237_0.9276.pth.tar

[239/300] divergence: 0.925    D: 0.105    D_real: 0.105    D_fake: 0.105    G:4.92    
checkpoint >> checkpoints/anogan_238_0.925.pth.tar

[240/300] divergence: 0.924    D: 0.104    D_real: 0.0998   D_fake: 0.104    G:4.76    
checkpoint >> checkpoints/anogan_239_0.9234.pth.tar

[241/300] divergence: 0.929    D: 0.0988   D_real: 0.0848   D_fake: 0.0988   G:4.87    
checkpoint >> checkpoints/anogan_240_0.9282.pth.tar

[242/300] dive

## Anomaly Score

In [ ]:
# Define latent space (noise)
noise = Variable(torch.randn((2, 100, 1, 1)).cuda(), requires_grad=True)
noise_optimizer = optim.Adam([noise])

real = Variable(real_images[0:2].cuda())
fake = G(noise)

loss_a = anomaly_loss(G, D, real, fake)
loss_a.backward()
noise_optimizer.step()

## Loss Visualization

In [ ]:
def smooth(x,window_len=100,window='hanning'):
    x = np.array(x)
    
    if x.ndim != 1:
        raise ValueError("smooth only accepts 1 dimension arrays.")

    if x.size < window_len:
        raise ValueError("Input vector needs to be bigger than window size.")

    if window_len<3:
        return x

    if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
        raise ValueError("Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'")

    s=numpy.r_[x[window_len-1:0:-1],x,x[-1:-window_len:-1]]
    
    if window == 'flat': #moving average
        w=numpy.ones(window_len,'d')
    else:
        w=eval('numpy.'+window+'(window_len)')

    y=numpy.convolve(w/w.sum(),s,mode='valid')
    return y

In [ ]:
figsize(16, 6)
plot(smooth(d_real_losses), label='D(x) Loss')
plot(smooth(d_fake_losses), label='D(G(z)) Loss')
plot(smooth(g_losses), label='G(z) loss')
plot(smooth(divergences), label='divergences')

grid()
legend()

## Generator Evaluation

In [ ]:
noise = Variable(torch.randn((_batch_size, 100, 1, 1)).cuda())
images = 1-G(noise).view((-1, 28, 28)).data.cpu().numpy()

imshow(images[0], cmap=cm.gray_r)

fig, subplots = pylab.subplots(5, 10) # subplots(y축, x축 갯수)
fig.set_figheight(6)
fig.set_figwidth(10)

idx = 0
for _subs in subplots:
    for subplot in _subs:
        d = images[idx]
        subplot.get_xaxis().set_visible(False)
        subplot.get_yaxis().set_visible(False)
        subplot.imshow(d, cmap=cm.gray_r)
        idx += 1